<a href="https://colab.research.google.com/github/Pritam-Mondal18/Ai_Comic_Crafter/blob/main/Ai_Comic_Crafter_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.8 MB/s eta 0:00:00


In [2]:
pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.8-cp311-cp311-linux_x86_64.whl size=6008054 sha256=d2bf5784ee0038e5c96b8b1711e609fa599630c3eef2a62a984580e6ac1469f6
  Stored in directory: /root/.cache/pip/wheels/c0/03/66/eb3810eafd55d921b2be32896d1f44313996982360663aa80b
Successfully built llama-cpp-python


In [3]:
from huggingface_hub import login
login("YOUR_ACCESS-KEY")


In [4]:
%%writefile app.py
import streamlit as st
import torch
from diffusers import StableDiffusionXLPipeline
from transformers import pipeline
import os
import zipfile
from io import BytesIO
from PIL import Image
import numpy as np

# Set Page Configuration
st.set_page_config(page_title="AI Comic Crafter", page_icon="🤺", layout="wide")

# Enhanced Title Section
st.markdown(
    """
    <h1 style='text-align: center; color: #FF5733;'>🎨 AI Comic Crafter 🦸</h1>
    <p style='text-align: center; font-size:18px;'>Turn your ideas into stunning comic stories with AI!</p>
    """,
    unsafe_allow_html=True
)

# Load Stable Diffusion Model (Hugging Face Diffusers)
@st.cache_resource
def load_sd_model():
    model_id = "stabilityai/stable-diffusion-xl-base-1.0"
    pipe = StableDiffusionXLPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
    )
    pipe.to("cuda" if torch.cuda.is_available() else "cpu")
    return pipe

pipe = load_sd_model()

# Load a Smaller LLM Model (GPT-2 for Text Generation)
@st.cache_resource
def load_llm():
    return pipeline("text-generation", model="gpt2")

llm = load_llm()

# Sidebar Settings with Better UI
theme_color = "#FF5733"
st.sidebar.markdown(f"<h2 style='color: {theme_color};'>📜 Story Settings</h2>", unsafe_allow_html=True)
genre = st.sidebar.selectbox("Choose a Genre", ["Superhero", "Fantasy", "Sci-Fi", "Mystery"])
style = st.sidebar.selectbox("Comic Style", ["Manga", "Cartoon", "Realistic", "Vintage"])
collage_format = st.sidebar.selectbox("Collage Format", ["2x2 Grid", "Vertical Strip", "Horizontal Strip"])
num_panels = 4  # Fixed to match the 4 story sections

# Story Input
st.markdown("<h3>✍️ Write Your Comic Story</h3>", unsafe_allow_html=True)
introduction = st.text_area("Introduction", "")
storyline = st.text_area("Storyline", "")
climax = st.text_area("Climax", "")
moral = st.text_area("Moral", "")

# Generate Story Using LLM
if st.button("🤖 Generate Story with AI", help="Let AI generate a comic story for you!"):
    with st.spinner("Generating story elements..."):
        intro_text = llm(f"Generate an introduction for a {genre} comic story.", max_length=50)[0]['generated_text']
        story_text = llm(f"Generate the main storyline for a {genre} comic story.", max_length=100)[0]['generated_text']
        climax_text = llm(f"Generate a climax for a {genre} comic story.", max_length=50)[0]['generated_text']
        moral_text = llm(f"Generate a moral for a {genre} comic story.", max_length=40)[0]['generated_text']

    introduction = st.text_area("Introduction", intro_text)
    storyline = st.text_area("Storyline", story_text)
    climax = st.text_area("Climax", climax_text)
    moral = st.text_area("Moral", moral_text)

# Function to Generate Comic-Style Images
def generate_comic_images(prompts):
    images = []
    for prompt in prompts:
        panel_prompt = f"{prompt}, {style} comic-style, highly detailed, vibrant colors"
        image = pipe(prompt=panel_prompt).images[0]  # Generate image
        images.append(image)
    return images

# Create Collage
def create_collage(images, format_type):
    widths, heights = zip(*(img.size for img in images))

    if format_type == "2x2 Grid":
        collage_width = max(widths) * 2
        collage_height = max(heights) * 2
        collage = Image.new("RGB", (collage_width, collage_height))
        positions = [(0, 0), (max(widths), 0), (0, max(heights)), (max(widths), max(heights))]
    elif format_type == "Vertical Strip":
        collage_width = max(widths)
        collage_height = sum(heights)
        collage = Image.new("RGB", (collage_width, collage_height))
        positions = [(0, sum(heights[:i])) for i in range(len(images))]
    else:  # Horizontal Strip
        collage_width = sum(widths)
        collage_height = max(heights)
        collage = Image.new("RGB", (collage_width, collage_height))
        positions = [(sum(widths[:i]), 0) for i in range(len(images))]

    for img, pos in zip(images, positions):
        collage.paste(img, pos)

    return collage

# Generate Story & Images
if st.button("🤺 Generate Comic", help="Create your AI-powered comic panels!"):
    with st.spinner("Generating Comic Panels... 🖌️"):
        images = generate_comic_images([introduction, storyline, climax, moral])
        collage = create_collage(images, collage_format)
        collage_path = "comic_collage.png"
        collage.save(collage_path)

    st.image(collage_path, caption="🎭 Your AI-Generated Comic Collage! 🎭")

    # Zip and provide download button
    zip_buffer = BytesIO()
    with zipfile.ZipFile(zip_buffer, "w") as zip_file:
        zip_file.write(collage_path)
    zip_buffer.seek(0)
    st.sidebar.download_button(
        label="💽 Download Comic Collage",
        data=zip_buffer,
        file_name="comic_collage.zip",
        mime="application/zip"
    )

# Footer
st.sidebar.markdown("<hr>", unsafe_allow_html=True)
st.sidebar.markdown("✨ <b>Powered by AI & Comic-Builders 🚀</b>", unsafe_allow_html=True)

Writing app.py


In [5]:
!pkill -f ngrok


In [7]:
!ngrok authtoken YOUR_ACCESS-KEY

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
from pyngrok import ngrok
import os

# Run Streamlit in the background
os.system("streamlit run app.py &")

# Expose port 8501 (Streamlit's default port)
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

Streamlit app is live at: NgrokTunnel: "https://f5db-34-125-144-95.ngrok-free.app" -> "http://localhost:8501"
